In [4]:
import googlemaps
import math
import numpy as np

# Set up google maps connection with your personal API key
gmaps = googlemaps.Client(key='')

In [30]:
# Helps convert nautical miles to degrees latitude
def deg_long_per_nautical_at_lat(lat):
    # https://stackoverflow.com/questions/7825415/convert-nautical-miles-to-degrees-longitude-given-a-latitude-using-google-maps
    lat = lat * math.pi / 180
    const1, const2, const3 = 111412.84, -93.5, 0.118
    long_len = const1 * math.cos(lat) + const2 * math.cos(3 * lat) + const3 * math.cos(5 * lat)
    long_len = long_len * (3.280833333) / (5280 * 1.15077945)
    return 1 / long_len

# Generates a list of x,y,z values of terrain centered at a given coordinate
def gen_elevation_map(coord, dim):
    latitude, longitude = coord
    width, height = dim
    half_width, half_height = width//2, height//2

    # Loop through the grid
    input_map = []
    for row in range(-half_height, half_height+1):
        for col in range(-half_width, half_width+1):
            # Calculate the latitude and longitude for each point in our square grid
            new_lat = latitude + row / 60 / 20 # 20th of a mile
            input_map.append((new_lat, longitude + col * deg_long_per_nautical_at_lat(new_lat) / 20))

    # Google maps API rate limits at 270 coordinates per request
    # This code breaks up input into batches <= 250 coordinates in size
    res = []
    for i in range(math.ceil(len(input_map)/250)):
        idx = i*250
        next_batch = input_map[idx: min(idx + 250, len(input_map))]
        res = res + gmaps.elevation(next_batch)

    # Reshapes long list of elevations to square
    output_map = np.array(list(map(lambda x:x["elevation"], res))).reshape(width, height, 1)

    # Converts square back into long list 
    # This time with x,y,z coordinates
    XX,YY = np.meshgrid(np.arange(output_map.shape[1]),np.arange(output_map.shape[0]))
    temp_coords = np.vstack((output_map.ravel(),XX.ravel(),YY.ravel())).T
    temp_coords[:,1] -= half_height
    temp_coords[:,2] -= half_width 
    temp_coords[:,[0,1]] = temp_coords[:,[1,0]]

    return temp_coords

In [37]:
# Generate a map for mount everest
coord = (52.163398, -117.193483)
dim = (301, 301)
geo_map = gen_elevation_map(coord, dim)

geo_map

array([[-150.        , 1159.96643066, -150.        ],
       [-149.        , 1189.86584473, -150.        ],
       [-148.        , 1229.14282227, -150.        ],
       ...,
       [ 148.        , 2671.48242188,  150.        ],
       [ 149.        , 2680.39770508,  150.        ],
       [ 150.        , 2697.94604492,  150.        ]])

In [38]:
geo_map[:, 1] = geo_map[:, 1] / 100

_, everest_max, _ = geo_map.max(axis=0)
_, everest_min, _ = geo_map.min(axis=0)

with open("new_data.js", 'w') as f:
    f.write("export const everest_min = %f;\n"%everest_min)
    f.write("export const everest_max = %f;\n"%everest_max)
    f.write("export const everest = [\n")
    for row in geo_map:
        f.write("[%f,%f,%f],\n"%(row[0], row[1], row[2]))
    f.write("];\n")



gmaps.elevation(coord)